In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

data = pd.read_csv("Final Fashion Dataset.csv")

print("Initial data info:")
print(data.info())
print("\nFirst few rows:\n", data.head())

data = data.dropna()

if 'ratings' in data.columns:
    data = data[data['ratings'].notnull()]

if 'ratings' in data.columns:
    scaler = MinMaxScaler()
    data['ratings'] = scaler.fit_transform(data[['ratings']])

if data['user_id'].dtype == object:
    data['user_id'] = data['user_id'].astype('category')
if data['id'].dtype == object:
    data['id'] = data['id'].astype('category')

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
data['user_id'] = user_encoder.fit_transform(data['user_id'])
data['id'] = item_encoder.fit_transform(data['id'])

num_users = data['user_id'].nunique()
num_items = data['id'].nunique()

print("\nProcessed data info:")
print(data.info())
print("\nFirst few rows of processed data:\n", data.head())
print("\nNumber of unique users:", num_users)
print("Number of unique items:", num_items)


Initial data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36484 entries, 0 to 36483
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             36484 non-null  int64  
 1   id                  36484 non-null  int64  
 2   gender              36484 non-null  object 
 3   masterCategory      36484 non-null  object 
 4   subCategory         36484 non-null  object 
 5   articleType         36484 non-null  object 
 6   baseColour          36484 non-null  object 
 7   season              36484 non-null  object 
 8   year                36484 non-null  float64
 9   usage               36484 non-null  object 
 10  productDisplayName  36484 non-null  object 
 11  filename            36484 non-null  object 
 12  link                36484 non-null  object 
 13  ratings             36484 non-null  int64  
 14  review              36484 non-null  object 
 15  Month               36484 non-null

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd


df = pd.read_csv('Final Fashion Dataset.csv')

print(df.info())


embedding_dim = 50

num_users = df['user_id'].nunique()
num_items = df['id'].nunique()

user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_embedding')(user_input)
user_embedding = Flatten()(user_embedding)

item_input = Input(shape=(1,), name='item_input')
item_embedding = Embedding(input_dim=num_items, output_dim=embedding_dim, name='item_embedding')(item_input)
item_embedding = Flatten()(item_embedding)

combined = Concatenate()([user_embedding, item_embedding])

dense = Dense(128, activation='relu')(combined)
dense = Dropout(0.3)(dense)
dense = Dense(64, activation='relu')(dense)
dense = Dropout(0.3)(dense)
dense = Dense(32, activation='relu')(dense)

output = Dense(1, activation='sigmoid', name='output')(dense)

model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

model.summary()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36484 entries, 0 to 36483
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             36484 non-null  int64  
 1   id                  36484 non-null  int64  
 2   gender              36484 non-null  object 
 3   masterCategory      36484 non-null  object 
 4   subCategory         36484 non-null  object 
 5   articleType         36484 non-null  object 
 6   baseColour          36484 non-null  object 
 7   season              36484 non-null  object 
 8   year                36484 non-null  float64
 9   usage               36484 non-null  object 
 10  productDisplayName  36484 non-null  object 
 11  filename            36484 non-null  object 
 12  link                36484 non-null  object 
 13  ratings             36484 non-null  int64  
 14  review              36484 non-null  object 
 15  Month               36484 non-null  object 
 16  Pric

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ item_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ user_embedding (Embedding)    │ (None, 1, 50)             │         244,550 │ user_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ item_embedding (Embedding)    │ (None, 1, 50)             │       1,824,200 │ item_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 50)                │               0 │ user_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 50)                │               0 │ item_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 100)               │               0 │ flatten[0][0],             │
│                               │                           │                 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │          12,928 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │           8,256 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 64)                │               0 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 32)                │           2,080 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output (Dense)                │ (None, 1)                 │              33 │ dense_2[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,092,047 (7.98 MB)

 Trainable params: 2,092,047 (7.98 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Embedding, Flatten, Concatenate, Dense, Dropout, Input
from tensorflow.keras.models import Model

def create_outfit_recommendation_model(num_users, num_items, embedding_dim=50):
    product_input = Input(shape=(1,), name='product_input')
    
    product_embedding = Embedding(num_items, embedding_dim, name='product_embedding')(product_input)
    
]    product_vec = Flatten()(product_embedding)
    
    dense = Dense(128, activation='relu')(product_vec)
    dropout = Dropout(0.2)(dense)
    dense = Dense(64, activation='relu')(dropout)
    dropout = Dropout(0.2)(dense)
    dense = Dense(32, activation='relu')(dropout)
    
    bottomwear_output = Dense(1, activation='linear', name='bottomwear_output')(dense)
    shoes_output = Dense(1, activation='linear', name='shoes_output')(dense)
    accessories_output = Dense(1, activation='linear', name='accessories_output')(dense)
    
    model = Model(inputs=[product_input], outputs=[bottomwear_output, shoes_output, accessories_output])
    model.compile(optimizer='adam', loss='mse')
    
    return model

num_users = 1000  
num_items = 5000  

# Create the model
model = create_outfit_recommendation_model(num_users, num_items)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_input (InputLayer)    │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ product_embedding (Embedding) │ (None, 1, 50)             │         250,000 │ product_input[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_2 (Flatten)           │ (None, 50)                │               0 │ product_embedding[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 128)               │           6,528 │ flatten_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_2 (Dropout)           │ (None, 128)               │               0 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 64)                │           8,256 │ dropout_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_3 (Dropout)           │ (None, 64)                │               0 │ dense_4[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_5 (Dense)               │ (None, 32)                │           2,080 │ dropout_3[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bottomwear_output (Dense)     │ (None, 1)                 │              33 │ dense_5[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ shoes_output (Dense)          │ (None, 1)                 │              33 │ dense_5[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ accessories_output (Dense)    │ (None, 1)                 │              33 │ dense_5[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 266,963 (1.02 MB)

 Trainable params: 266,963 (1.02 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
print(df.columns)


Index(['user_id', 'id', 'gender', 'masterCategory', 'subCategory',
       'articleType', 'baseColour', 'season', 'year', 'usage',
       'productDisplayName', 'filename', 'link', 'ratings', 'review', 'Month',
       'Price (USD)'],
      dtype='object')


In [12]:
df.head()


,user_id,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link,ratings,review,Month,Price (USD)
0,93810,8493,Women,Apparel,Bottomwear,Skirts,Black,Fall,2011.0,Casual,Forever New Women Black Skirts,8493.jpg,http://assets.myntassets.com/v1/images/style/p...,5,Amazing quality! Definitely recommend. Would r...,November,46
1,24592,30757,Men,Apparel,Topwear,Kurtas,Grey,Summer,2012.0,Ethnic,Fabindia Men Grey Mangalgiri Kurta,30757.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Perfect for any occasion, great buy..",September,29
2,13278,14881,Women,Accessories,Bags,Handbags,Black,Summer,2011.0,Casual,United Colors of Benetton Women Solid Black Ha...,14881.jpg,http://assets.myntassets.com/v1/images/style/p...,3,"It's okay, nothing special.!",October,39
3,46048,48449,Men,Apparel,Topwear,Tshirts,Blue,Summer,2012.0,Casual,French Connection Men Blue T-shirt,48449.jpg,http://assets.myntassets.com/v1/images/style/p...,5,"Comfortable and stylish, worth the price. and ...",August,39
4,42098,4697,Unisex,Accessories,Watches,Watches,Black,Winter,2016.0,Sports,ADIDAS Unisex Digital Duramo Black Watch,4697.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Perfect for any occasion, great buy.!",March,33


In [24]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd

df = pd.read_csv('Final Fashion Dataset.csv')

label_encoder_gender = LabelEncoder()
df['gender_encoded'] = label_encoder_gender.fit_transform(df['gender'])
gender_mapping = {label: idx for label, idx in zip(label_encoder_gender.classes_, label_encoder_gender.transform(label_encoder_gender.classes_))}
print("Gender Mapping:", gender_mapping)

label_encoder_color = LabelEncoder()
df['baseColour_encoded'] = label_encoder_color.fit_transform(df['baseColour'])
colour_mapping = {label: idx for label, idx in zip(label_encoder_color.classes_, label_encoder_color.transform(label_encoder_color.classes_))}
print("Colour Mapping:", colour_mapping)

label_encoder_usage = LabelEncoder()
df['usage_encoded'] = label_encoder_usage.fit_transform(df['usage'])
usage_mapping = {label: idx for label, idx in zip(label_encoder_usage.classes_, label_encoder_usage.transform(label_encoder_usage.classes_))}
print("Usage Mapping:", usage_mapping)


label_encoder_category = LabelEncoder()
df['masterCategory_encoded'] = label_encoder_category.fit_transform(df['masterCategory'])
category_mapping = {label: idx for label, idx in zip(label_encoder_category.classes_, label_encoder_category.transform(label_encoder_category.classes_))}
print("MasterCategory Mapping:", category_mapping)

label_encoder_subcategory = LabelEncoder()
df['subCategory_encoded'] = label_encoder_subcategory.fit_transform(df['subCategory'])
subcategory_mapping = {label: idx for label, idx in zip(label_encoder_subcategory.classes_, label_encoder_subcategory.transform(label_encoder_subcategory.classes_))}
print("SubCategory Mapping:", subcategory_mapping)

one_hot_encoder = OneHotEncoder(sparse_output=False)
season_encoded = one_hot_encoder.fit_transform(df[['season']])
season_columns = one_hot_encoder.get_feature_names_out(['season'])
df = df.join(pd.DataFrame(season_encoded, columns=season_columns))

season_mapping = {label: idx for idx, label in enumerate(one_hot_encoder.categories_[0])}
print("Season Mapping:", season_mapping)

df.drop(['gender', 'baseColour', 'usage', 'masterCategory', 'subCategory', 'season'], axis=1, inplace=True)

df.head()


Gender Mapping: {'Boys': 0, 'Girls': 1, 'Men': 2, 'Unisex': 3, 'Women': 4}
Colour Mapping: {'Beige': 0, 'Black': 1, 'Blue': 2, 'Bronze': 3, 'Brown': 4, 'Burgundy': 5, 'Charcoal': 6, 'Coffee Brown': 7, 'Copper': 8, 'Cream': 9, 'Fluorescent Green': 10, 'Gold': 11, 'Green': 12, 'Grey': 13, 'Grey Melange': 14, 'Khaki': 15, 'Lavender': 16, 'Lime Green': 17, 'Magenta': 18, 'Maroon': 19, 'Mauve': 20, 'Metallic': 21, 'Multi': 22, 'Mushroom Brown': 23, 'Mustard': 24, 'Navy Blue': 25, 'Nude': 26, 'Off White': 27, 'Olive': 28, 'Orange': 29, 'Peach': 30, 'Pink': 31, 'Purple': 32, 'Red': 33, 'Rose': 34, 'Rust': 35, 'Sea Green': 36, 'Silver': 37, 'Steel': 38, 'Tan': 39, 'Taupe': 40, 'Teal': 41, 'Turquoise Blue': 42, 'White': 43, 'Yellow': 44}
Usage Mapping: {'Casual': 0, 'Ethnic': 1, 'Formal': 2, 'Party': 3, 'Smart Casual': 4, 'Sports': 5, 'Travel': 6}
MasterCategory Mapping: {'Accessories': 0, 'Apparel': 1, 'Footwear': 2}
SubCategory Mapping: {'Bags': 0, 'Belts': 1, 'Bottomwear': 2, 'Eyewear': 3, '

,user_id,id,articleType,year,productDisplayName,filename,link,ratings,review,Month,Price (USD),gender_encoded,baseColour_encoded,usage_encoded,masterCategory_encoded,subCategory_encoded,season_Fall,season_Spring,season_Summer,season_Winter
0,93810,8493,Skirts,2011.0,Forever New Women Black Skirts,8493.jpg,http://assets.myntassets.com/v1/images/style/p...,5,Amazing quality! Definitely recommend. Would r...,November,46,4,1,0,1,2,1.0,0.0,0.0,0.0
1,24592,30757,Kurtas,2012.0,Fabindia Men Grey Mangalgiri Kurta,30757.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Perfect for any occasion, great buy..",September,29,2,13,1,1,10,0.0,0.0,1.0,0.0
2,13278,14881,Handbags,2011.0,United Colors of Benetton Women Solid Black Ha...,14881.jpg,http://assets.myntassets.com/v1/images/style/p...,3,"It's okay, nothing special.!",October,39,4,1,0,0,0,0.0,0.0,1.0,0.0
3,46048,48449,Tshirts,2012.0,French Connection Men Blue T-shirt,48449.jpg,http://assets.myntassets.com/v1/images/style/p...,5,"Comfortable and stylish, worth the price. and ...",August,39,2,2,0,1,10,0.0,0.0,1.0,0.0
4,42098,4697,Watches,2016.0,ADIDAS Unisex Digital Duramo Black Watch,4697.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Perfect for any occasion, great buy.!",March,33,3,1,5,0,12,0.0,0.0,0.0,1.0
